In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import DataLoader
import numpy as np 

from models import MnistNet
from utils import train, test
from create_dataset import create_iid_dataset
from utils import load_partition, load_data
import copy

In [3]:
DATA_DIR = './datasets/fl_mnist_noniid/'
num_clients = 2
B = 32
# Train loaders for clients
train_sets = [load_partition(cid=x + 1, data_dir=DATA_DIR) for x in range(num_clients)]
train_loaders = [DataLoader(dataset, batch_size=B, shuffle=True) for dataset in train_sets]
_, test_loader = load_data()

/Users/htnu/Research/Flower_FL_examples/utils.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train).type(torch.FloatTensor)


In [4]:
[len(train_sets[i]) for i in range(num_clients)]

[30000, 30000]

In [5]:
device = 'mps'
net_glob = MnistNet()
net_glob.train()
w_glob = net_glob.state_dict()

In [6]:
loss_train = []
cv_loss, cv_acc = [], []
val_loss_pre, counter = 0, 0
net_best = None
best_loss = None 
val_acc_list, net_list = [], []

In [7]:
round = 10
local_epoch = 5
lr = 0.01

In [8]:
def FedAvg(w):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[k] += w[i][k]
        w_avg[k] = torch.div(w_avg[k], len(w))
    return w_avg

In [9]:
w_locals = [w_glob for _ in range(num_clients)]
val_acc_list, val_loss_list = [], []
for iter in range(round):
    loss_locals = []
    idxs_clients = list(range(num_clients))
    round_lr = lr/(1 + iter) # weight decay for lr at each round
    # local_epoch += 10
    for idx in idxs_clients:
        print(f'Client {idx} is training')
        train_local_loader = train_loaders[idx]
        net_local = copy.deepcopy(net_glob).to(device)
        train(net_local, train_local_loader, local_epoch, device, lr=round_lr)
        w_locals[idx] = net_local.state_dict()
        print('===============')

    w_glob = FedAvg(w_locals)
    net_glob.load_state_dict(w_glob)
    print(f'Round {iter + 1} - Centralize Evaluation')
    loss, acc, total = test(net_glob, test_loader, device)
    val_loss_list.append(loss)
    val_acc_list.append(acc)
    print(f'Num samples {total} | Loss {loss} | Acc {acc}')

Client 0 is training


 20%|██        | 1/5 [00:11<00:47, 11.84s/it]

Epoch 1: train loss inf, accuracy 0.3257333333333333


 40%|████      | 2/5 [00:23<00:35, 11.70s/it]

Epoch 2: train loss 0.03894336133797963, accuracy 0.398


...

 80%|████████  | 4/5 [00:35<00:08,  8.93s/it]

Epoch 4: train loss 0.008979927549511194, accuracy 0.9131


100%|██████████| 5/5 [00:44<00:00,  8.94s/it]

Epoch 5: train loss 0.008964624838158488, accuracy 0.9134
Round 10 - Centralize Evaluation


Num samples 10000 | Loss 0.07438940155506134 | Acc 0.1538
